In [1]:
import pandas as pd
import numpy as np
import re
from difflib import SequenceMatcher    # https://docs.python.org/3/library/difflib.html#difflib.get_close_matches
# from cleanco import cleanco   # would ne useful

# Unternehmensliste Interflex

In [2]:
ulist = pd.read_excel("Unternehmensliste_Interflex.xlsx")
ulist.head()

,Firmenname,Land,Postleitzahl,Ort,Straße und Hausnr.,USt.-IdNr.,ID
0,Bistum Essen,Deutschland,45127,Essen,Zwölfling 16,NaN,801695
1,KBS Kleider Bauer Betriebs-GmbH,Österreich,2380,Perchtoldsdorf,Zwingenstr. 5,NaN,850801
2,Elektro Ing-Plan GmbH Dresden,Deutschland,1187,Dresden,Zwickauer Straße 88,NaN,851014
3,Netzdesign-Vobornik,Deutschland,71083,Herrenberg,Zwickauer Strasse 41,NaN,NaN
4,NILES-SIMMONS Industrieanlagen GmbH,Deutschland,9117,Chemnitz,Zwickauer Straße 355,DE140853999,802541


In [3]:
ulist = ulist.assign(Idtrack=lambda ulist: range(1, len(ulist)+1))
ulist_original = ulist.copy()

**Aim is to remove suffix and rest** ==> E.g: Sensient Imaging Technologies GmbH Chemiepark Bitterfeld-Wolfen

In [4]:
ulist["Firmenname"] = ulist["Firmenname"].str.lower()   # faster
# ulist[ulist['Firmenname'].str.endswith(tuple(suffix_list))] some finishes with gmbh some -gmbh and some even continues

In [5]:
# Suffix list for company names   (ordering is important! Will be fixed)
suffix_list = ['gmbh', 'gmbh&co.', 'ggmbh', 'gmbh+co.', 'kg-gmbh', 'gmbh&co', 'ltd.', 'se',
      'gmbh&co.kg', 'gmbh&cokg', 'gmbh.', 'gmbh,', 'gmbh&c'
               'company', 'incorporated', 'corporation', 'corp.', 'corp', 'inc',
      '& co.', '& co',  'inc.', 's.p.a.', 'n.v.', 'a.g.', 'ag', 'nuf', 's.a.', 's.f.',
      'oao', 'co.', 'co',
              'soc.col.', 'stg', 'd.n.o.', 'ltda.', 'v.o.s.', 'a spol.',
      u've\xc5\x99. obch. spol.', 'kgaa', 'o.e.', 's.f.', 's.n.c.', 's.a.p.a.', 'j.t.d.',
      'v.o.f.', 'sp.j.', 'og', 'sd', ' i/s', 'ay', 'snc', 'oe', 'bt.', 's.s.', 'mb',
      'ans', 'da', 'o.d.', 'hb', 'pt',
              'unltd', 'ultd', 'sal', 'unlimited', 'saog', 'saoc', 'aj',
      'yoaj', 'oaj', 'akc. spol.', 'a.s.',
              'esv', 'gie', 'kv.', 'qk',
              'pty. ltd.', 'pty ltd', 'ltd', 'l.t.d.', 'bvba', 'd.o.o.', 'ltda', 'gmbh',
      'g.m.b.h', 'kft.', 'kht.', 'zrt.', 'ehf.', 's.a.r.l.', 'd.o.o.e.l.', 's. de r.l.',
      'b.v.', 'tapui',
      'sp. z.o.o.', 'sp. z o.o.', 'spółka z o.o.',
      's.r.l.', 's.l.', 's.l.n.e.', 'ood', 'oy', 'rt.',
      'teo', 'uab', 'scs', 'sprl', 'limited', 'bhd.', 'sdn. bhd.', 'sdn bhd', 'as',
      'lda.', 'tov', 'pp',
              'pllc', 'llc', 'l.l.c.', 'plc.', 'plc', 'hf.', 'oyj',
      'a.e.', 'nyrt.', 'p.l.c.', 'sh.a.', 's.a.', 's.r.l.', 'srl.', 'srl', 'aat', '3at', 'd.d.',
      's.r.o.', 'spol. s r.o.', 's.m.b.a.', 'smba', 'sarl', 'nv', 'sa', 'aps',
      'a/s', 'p/s', 'sae', 'sasu', 'eurl', 'ae', 'cpt', 'as', 'ab', 'asa', 'ooo', 'dat',
      'vat', 'zat', 'mchj', 'a.d.',
              'lllp', 'l.l.l.p.',
              'llp', 'l.l.p.', 'sp.p.', 's.c.a.', 's.c.s.',
              'gmbh & co. kg', 'lp', 'l.p.', 's.c.s.',
      's.c.p.a', 'comm.v', 'k.d.', 'k.d.a.', 's. en c.', 'e.e.', 's.a.s.', 's. en c.',
      'c.v.', 's.k.a.', 'sp.k.', 's.cra.', 'ky', 'scs', 'kg', 'kd', 'k/s', 'ee', 'secs',
      'kda', 'ks', 'kb','kt',
              'sicav',
              'nl',
              'vzw', 'ses.', 'gte.',
              'private', 'pte', 'xk',
              'p.c.', 'vof', 'snc',
              'pllc', 'p.l.l.c.',
              'e.u.', 's.p.', 't:mi', 'tmi', 'e.v.', 'e.c.', 'et', 'obrt',
      'fie', 'ij', 'fop', 'xt']

# Source of company suffices https://github.com/psolin/cleanco/blob/master/cleanco/termdata.py

In [6]:
#suffix_list += ['-{}'.format(x) for x in suffix_list]   # add dash symbol in front of suffices
#city_names =    # Bistum Essen ==> Bistum?

In [7]:
def suffix_remover(companyname):
    for i in suffix_list:
        #if i in re.split(' |-', companyname):   # doesn't work properly
        if i in companyname.split():    # tokenization
            return companyname[:companyname.index(i)-1]
        elif i in companyname.split('-'):
            return companyname[:companyname.index(i)-1]
    return companyname

In [8]:
ulist['Firmenname'] = ulist['Firmenname'].apply(suffix_remover)
ulist['Firmenname'].head()

0                      bistum essen
1        kbs kleider bauer betriebs
2                  elektro ing-plan
3               netzdesign-vobornik
4    niles-simmons industrieanlagen
Name: Firmenname, dtype: object

In [11]:
print('Remaining suffix as gmbh (most common):', len(ulist[ulist['Firmenname'].str.contains('gmbh')]))

Remaining suffix as gmbh (most common): 43


In [12]:
#[x for x in ulist['Firmenname'] if any(i in x for i in suffix_list)]

# https://stackoverflow.com/questions/4843158/how-to-check-if-a-string-is-a-substring-of-items-in-a-list-of-strings

# def remafterellipsis(text):
#     for i in suffix_list:
#         wher = text.find(i)
#         if wher != -1:
#             #print(i)
#             #print(text[:wher])
#             return text[:wher]
#     return text
# 
# ulist['Firmenname'].apply(remafterellipsis)

In [13]:
ulist.iloc[:,:-1].duplicated().sum() 
#need to remove Idtracker column to find duplicates because it increments for each entry

15

In [14]:
ulist.nunique()

Firmenname            13548
Land                     73
Postleitzahl           4946
Ort                    4239
Straße und Hausnr.    12702
USt.-IdNr.             4427
ID                    10852
Idtrack               14550
dtype: int64

In [15]:
ulist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14550 entries, 0 to 14549
Data columns (total 8 columns):
Firmenname            14550 non-null object
Land                  13477 non-null object
Postleitzahl          14238 non-null object
Ort                   13902 non-null object
Straße und Hausnr.    14111 non-null object
USt.-IdNr.            4849 non-null object
ID                    10875 non-null object
Idtrack               14550 non-null int32
dtypes: int32(1), object(7)
memory usage: 852.7+ KB


In [16]:
ulist = (
    ulist
        .assign(
            Land=ulist['Land'].astype('category'),
            # Postleitzahl=ulist['Postleitzahl'].astype('Int64'),
            # ID=ulist['ID'].astype('Int64')   # not working
        )   # .info()
)

In [17]:
ulist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14550 entries, 0 to 14549
Data columns (total 8 columns):
Firmenname            14550 non-null object
Land                  13477 non-null category
Postleitzahl          14238 non-null object
Ort                   13902 non-null object
Straße und Hausnr.    14111 non-null object
USt.-IdNr.            4849 non-null object
ID                    10875 non-null object
Idtrack               14550 non-null int32
dtypes: category(1), int32(1), object(6)
memory usage: 756.3+ KB


In [18]:
# pipeline
# ulist.duplicated().sum()   # remove duplicated rows in the dataframe
ulist["Firmenname"] = ulist["Firmenname"].str.lower()   # faster
ulist["Firmenname"] = ulist["Firmenname"].str.replace('\W', '')      # special characters are removed
ulist["Land"] = ulist["Land"].str.upper()
ulist["Land"] = ulist["Land"].str.replace('\W', '').astype('category')
ulist["Ort"] = ulist["Ort"].str.lower()
ulist["Ort"] = ulist["Ort"].str.replace('\W', '')
ulist['USt.-IdNr.'] = ulist['USt.-IdNr.'].str.replace(" ","")        # white space removed
ulist['ID'] = ulist['ID'].astype('str').str.replace('[^0-9]', '')    # ID with only numbers
# ulist['ID'] = ulist['ID'].astype('str').replace(r'\D+', '', regex=True)
ulist['ID'] = ulist['ID'].replace(r'^\s*$', np.NaN, regex=True)      # to keep missing values instead of empty strings

In [19]:
# a = list(set(ulist[ulist.loc[:,['Firmenname', 'Land', 'Ort']].duplicated()].index) - set(ulist[ulist.loc[:,['Firmenname', 'Land', 'Postleitzahl', 'Ort']].duplicated()].index))
# a.sort()
# ulist.loc[a]

In [20]:
# moved NA of 'USt.-IdNr.' and 'ID' columns to bottom of df so that rows with more NA's will be removed because they are at the bottom.
sorted_duplicates = ulist.sort_values(by=["USt.-IdNr.", "ID"], na_position='last')      # move NAs to the bottom of the df
ulist = sorted_duplicates.drop_duplicates(subset=["Firmenname", "Land", "Ort"],
                                     keep="first").sort_index().reset_index(drop=True)  # drop duplicates

In [21]:
ulist[ulist["Firmenname"].str.startswith('rweiss')]   # Actually both belongs to R.WEISS Packaging GmbH & Co. KG

,Firmenname,Land,Postleitzahl,Ort,Straße und Hausnr.,USt.-IdNr.,ID,Idtrack
24,rweissmaschinenbau,DEUTSCHLAND,74564,crailsheim,ZUR FLÜGELAU 28-30,DE190976983,181817,26
25,rweissautomation,DEUTSCHLAND,74564,crailsheim,ZUR FLÜGELAU 28,NaN,803402,28


In [22]:
len(ulist)   # it was 14090 without removing suffices

13992

In [25]:
middle = ['stiftung', 'holding']    # can be found in the middle of a company name   .str.contains()

In [24]:
# sintmaartenskliniek   # city different but everything else same

In [29]:
ulist[ulist['Firmenname'].str.contains('gmbh')]

,Firmenname,Land,Postleitzahl,Ort,Straße und Hausnr.,USt.-IdNr.,ID,Idtrack
97,qsgmbhgenthin,DEUTSCHLAND,39307,genthin,Ziegeleistr. 56,DE139333681,150616,103
155,gfvgesellschaftfürvermögensverwaltungmbh,DEUTSCHLAND,45096,essen,Zentraler Rechnungseingang,NaN,804621,162
182,msgmbhbereichmanagement,DEUTSCHLAND,9130,chemnitz,Zeisigwaldstraße 101,NaN,160399,190
299,operationalservicesgmbh,DEUTSCHLAND,1307,NaN,Wintergartenstraße 4,NaN,141099,312
536,technimarkeisbärgmbhkunststoffundmetallverarb,DEUTSCHLAND,52477,NaN,Werner-von-Siemens-Straße 7-9,NaN,120800,555
...,...,...,...,...,...,...,...,...
11898,zevzwickauerenergieversgmbh,DEUTSCHLAND,8056,zwickau,Bahnhofstraße 4,DE141379256,802588,12385
12379,altmannböhninggmbhniederlassungdresden,DEUTSCHLAND,1458,NaN,An den Schindertannen 4,NaN,141001,12883
12842,erzgebirgsklinikumggmbhhauszschopau,DEUTSCHLAND,9405,zschopau,Alte Marienberger Str. 52,NaN,850387,13358
12968,sigmagrundstücksundverwaltungsgmbhvertretendur...,DEUTSCHLAND,82027,grünwald,Albert Immobilienverw.Ges.mbH,NaN,171092,13494


Next step is grouping duplicated entries within a unique company name. Then applying same methods to companies from other data sets. First, removing suffices and then searching duplicated companies in company name, country and city columns. My goal is to decrease the size of data sets by removing duplicates. After that, I will append remaining unique company names to a final dataframe or a dictionary. Later, I can come back and group the duplicated ones for each unique company and create a many-to-one mapping function.

At the end, inputing a company name will return a unique company name (+information too like country, city, revenue?) based on grouping same companies with varying names and a rule-based approach.

For example:
Input (Volkswagen) ==> Volkswagen AG   based on suffix (adding suffix and searching)

Input (Volkswagen Aktiengesellschaft) ==> Volkswagen AG    (based on grouping same companies)

Input (VW) ==> Volkswagen AG   based on a rule

In [38]:
#ulist[ulist['Firmenname'].str.contains('volkswagen')]

# My questions so far
1. This dataset includes mostly German companies, while other datasets have no German companies. So, there will be mostly one-to-one mapping from this dataset. Is it normal or are there companies from other datasets to be matched with this dataset even though they are in different countries?
2. Country names are in German in this dataset. Should I convert them to English?
3. Are USt.-IdNr. and ID columns in this dataset important? Or they will be dropped?
4. What should many-to-one function return along with a company name? Country, city etc?